# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Вводные данные:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Для начала избавимся от пропусков, дубликатов и артефактов в данных, а затем проанализируем их и ответим на вопросы.

### Шаг 1. Откроем файл с данными и изучим общую информацию. 

In [2]:
import pandas as pd
bank_clients = pd.read_csv('/datasets/data.csv')

In [3]:
bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
bank_clients.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Всего в таблице 12 столбцов с типами float, int и object. Количество значений в столбцах отличается, это значит, что в столбцах есть пропущенные значения. Некоторые данные представлены в вещественном типе, его нужно изменить на целочисленный для удобства подсчёта. 

Причинами появления пропусков могут быть как человеческий фактор, так и ошибки системы или процесса выгрузки. В данном случае можно отследить закономерность - в строках с пропущенными значениями days_employed также пропущены значения total_income. Вероятность пропуска зависит от других значений в наборе данных, но не от значений собственного столбца. Эти пропуски случайны.

### Шаг 2. Предобработка данных

### Обработка пропусков

В столбце days_employed найдены пропуски и артефакты - в некоторых случаях стаж отрицательный, в остальных он неправдоподобно большой. Скорее всего, база данных была ошибочно заполнена, либо данные были неправильно извлечены. Избавимся от этого столбца для удобства работы - для дальнейших расчетов он не понадобится.

Пропуски также найдены в столбце total_income. Этот столбец нам понадобится, поэтому заменим пропущенные значение средним арифметическим или медианой, чтобы не потерять другие данные. Так как речь идёт о доходе, для заполнения лучше использовать медиану, чтобы сильно выделяющиеся значения не повлияли на общий результат. Медиану рассчитаем отдельно для каждого типа дохода.

In [5]:
bank_clients.drop(['days_employed'], axis='columns', inplace=True)        

In [6]:
bank_clients['total_income'] = bank_clients['total_income'].fillna(-1)

In [7]:
def fill_income(row):
    total_income = row['total_income']
    income_type = row['income_type']
    if total_income == -1:      
        return income_type_to_median[income_type]
    return total_income
 
income_type_to_median = bank_clients.groupby("income_type").median()["total_income"].to_dict()    
       
bank_clients['total_income'] = bank_clients.apply(fill_income, axis=1)

bank_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 11 columns):
children            21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(1), int64(5), object(5)
memory usage: 1.8+ MB


Проверим столбец dob_years.

In [8]:
bank_clients['dob_years'].value_counts().tail(11)

0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В столбце есть нулевые значения, заменим их на среднее арифметическое - в случае возраста такой метод допустим.

In [9]:
bank_clients.loc[bank_clients['dob_years'] == 0] = int(bank_clients['dob_years'].mean())
bank_clients.loc[bank_clients['dob_years'] == 0].count().sum()

0

Проверим столбец children на реалистичность данных.

In [10]:
bank_clients['children'].value_counts()

 0     14080
 1      4802
 2      2042
 3       328
 43      101
 20       75
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце встречаются отрицательные и нереалистично большие значения. Доля неверных значений - меньше одного процента, на результат исследования они не повлияют. Избавимся от этих строчек для правильности подсчетов.

In [11]:
bank_clients = bank_clients.loc[bank_clients['children'] >= 0]
bank_clients = bank_clients.loc[bank_clients['children'] < 20]
bank_clients['children'].value_counts()

0    14080
1     4802
2     2042
3      328
4       41
5        9
Name: children, dtype: int64

### Вывод

В данных найдены пропуски и ошибки. Мы избавились от ошибочных данных, которые нам не понадобятся, заполнили NaN значения и изменили неправильные в данных, которые нужны. 

### Замена типа данных

Тип данных столбца total_income - float. Заменим тип данных на int с помощью метода .astype(), чтобы получить целочисленные значения.

In [12]:
try:
    bank_clients['total_income'] = int(bank_clients['total_income'])
except:
    bank_clients['total_income'] = bank_clients['total_income'].astype(int)
bank_clients['total_income'].dtype

dtype('int64')

### Вывод

Все числовые данные теперь типа int, можно переходить к обработке дубликатов.

### Обработка дубликатов

Значения в столбце education отличаются регистром. Применим к столбцу метод value_counts() для проверки, а после приведем значения к одному регистру.

In [13]:
bank_clients['education'].value_counts()

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [14]:
bank_clients['education'] = bank_clients['education'].str.lower()
bank_clients['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь установим наличие полных дубликатов и избавимся от них.

In [15]:
bank_clients.duplicated().sum()

71

In [16]:
bank_clients = bank_clients.drop_duplicates().reset_index(drop=True)
bank_clients.duplicated().sum()

0

### Вывод

Дубликаты в данных могли появиться в результате сбоев или неправильных скриптов. Мы использовали метод value_counts() для ознакомления со всеми значениями столбца education, привели их к одному регистру, а после удалили полные дубликаты используя метод drop_duplicates().

### Лемматизация

Ознакомимся со всеми уникальными значениями столбца purpose.

In [17]:
bank_clients['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      613
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

Вариаций целей получения кредита очень много, некоторые из них содержат орфографические ошибки, но их можно разделить на 4 категории - автомобиль, свадьба, образование и недвижимость. Лемматизируем значения purpose с помощью библиотеки pymystem3, а после добавим новый столбец purpose_category с категориями целей.

In [18]:
from pymystem3 import Mystem
m = Mystem()

def find_category(row):
    lemmas = m.lemmatize(row)
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование'in lemmas:
        return 'образование'
    return 'недвижимость'

bank_clients['purpose_category'] = bank_clients['purpose'].apply(find_category)

bank_clients.head(10)

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость
7,0,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость


### Вывод

Все цели получения кредита мы выделили в 4 группы - так будет намного удобнее анализировать данные.

### Категоризация данных

In [19]:
income_cut = pd.qcut(bank_clients['total_income'], 4)
income_cut.value_counts()

(195819.5, 2265604.0]    5308
(107555.0, 140235.0]     5308
(20666.999, 107555.0]    5308
(140235.0, 195819.5]     5307
Name: total_income, dtype: int64

С помощью метода .qcut() разбили данные  в столбце total_income на квантили.

Для ответа на вопрос о связи дохода клиента с вероятностью возникновения задолженности, разделим клиентов на 4 категории по уровню дохода с помощью функции find_income_group.

low, если доход < 107555 тыс.р

medium, если доход >= 107555 тыс.р и < 140235 тыс.р

high, если доход >= 140235 тыс.р и < 195819.5 тыс.р

high+, если доход >= 195819.5 тыс.р

In [20]:
def find_income_group(income):
    if income < 107555:
        return('low')
    if 107555 <= income <= 140235:
        return('medium')
    if 140235 <= income <= 195819.5:
        return('high')
    return('high+')

### Вывод

Разделили клиентов банка на 4 категории по уровню дохода.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [22]:
debt_and_children = bank_clients[['children', 'debt']]

clients_with_children = debt_and_children.loc[debt_and_children['children'] > 0]
debtors_with_children = clients_with_children.loc[clients_with_children['debt'] == 1]
debtors_with_children_mean = len(debtors_with_children) / len(clients_with_children)

clients_without_children = debt_and_children.loc[debt_and_children['children'] == 0]
debtors_without_children = clients_without_children.loc[clients_without_children['debt'] == 1]
debtors_without_children_mean = len(debtors_without_children) / len(clients_without_children)

if debtors_with_children_mean > debtors_without_children_mean:
    print('Доля должников с детьми - {:.1%}, это больше, чем доля должников без детей.'.format(debtors_with_children_mean))
else:
    print('Доля должников без детей - {:.1%}, это больше, чем доля должников с детьми.'.format(debtors_without_children_mean))

Доля должников с детьми - 9.2%, это больше, чем доля должников без детей.


### Вывод

Доля должников с детьми больше доли бездетных должников. Отсюда вывод, что наличие детей негативно влияет на возврат кредита в срок.

In [21]:
debtors_family_status = bank_clients[['family_status', 'debt']]
debtors_family_status_grouped = debtors_family_status.groupby('family_status').count()
debtors_family_status_grouped['number'] = debtors_family_status.loc[debtors_family_status['debt'] == 1].groupby('family_status').sum()
debtors_family_status_grouped['mean, %'] = debtors_family_status_grouped['number'] / debtors_family_status_grouped['debt'] * 100
debtors_family_status_grouped.sort_values('mean, %', ascending=False)

,debt,number,"mean, %"
family_status,,,
Не женат / не замужем,2780,272,9.784173
гражданский брак,4113,383,9.311938
женат / замужем,12213,923,7.557521
в разводе,1179,84,7.124682
вдовец / вдова,946,62,6.553911


### Вывод

Результат исследования показывает, что в группе клиентов со статусом *не женат / не замужем* доля должников больше всего - почти 9,8%. Меньше всего должников среди вдовцов - 6,5%.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [27]:
debtors_income = bank_clients[['total_income', 'debt']]
debtors_income = debtors_income.loc[debtors_income['debt'] == 1]
debtors_income['income_group'] = debtors_income['total_income'].apply(find_income_group)
debtors_income[['income_group', 'debt']].groupby('income_group').sum().sort_values('debt', ascending = False)

,debt
income_group,
high,463
medium,456
low,425
high+,380


### Вывод

Поделили клиентов на 4 категории (равных по количеству клиентов) по уровню дохода.
В результате подсчета должников в этих категориях видим, что больше всего таковых в категории дохода high. Однако, следует заметить, что количество должников в категориях отличается совсем немного, за исключением категории high+ - здесь их значительно меньше.

Как результат, можем сказать, что выраженной зависимости между уровнем дохода и возвратом кредита в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [23]:
bank_clients_purpose = bank_clients.pivot_table(index=['purpose_category', 'purpose'], values='debt', aggfunc='sum')
bank_clients_purpose.sort_values('debt', ascending=False).head(10)

debt
purpose_category purpose                                    
свадьба          свадьба                                  63
                 на проведение свадьбы                    61
                 сыграть свадьбу                          57
недвижимость     операции с недвижимостью                 55
                 строительство недвижимости               54
                 покупка жилья для сдачи                  52
                 операции с коммерческой недвижимостью    52
образование      получение дополнительного образования    51
автомобиль       сделка с подержанным автомобилем         51
                 сделка с автомобилем                     50

Основываясь на результатах этой таблицы, чаще всего должниками становятся в результате взятия кредита на проведение свадьбы и на покупку/строительство недвижимости. Но какими будут результаты, если сгрупировать данные по категории цели кредита и выявить долю должников в каждой из категорий?

In [44]:
debtors_purpose = bank_clients[['purpose_category', 'purpose', 'debt']]
debtors_purpose_grouped = debtors_purpose.groupby('purpose_category').count()
debtors_purpose_grouped['number'] = debtors_purpose.loc[debtors_purpose['debt'] == 1].groupby('purpose_category').sum()
debtors_purpose_grouped['mean, %'] = debtors_purpose_grouped['number'] / debtors_purpose_grouped['debt'] * 100
debtors_purpose_grouped[['mean, %']].sort_values('mean, %', ascending=False)

,"mean, %"
purpose_category,
автомобиль,9.323626
образование,9.294710
свадьба,7.872988
недвижимость,7.258969


### Вывод

В результате группирования по категории цели кредита видим, что количество должников в категориях "автомобиль" и "образование" выделяется в большую сторону. И, хотя при первой проверке сильно выделялись категории "свадьба" и "недвижимость", при использовании относительных величин картина изменилась.

### Шаг 4. Общий вывод

В ходе работы искали зависимость между:

1. наличием детей и возвратом кредита в срок - должников с детьми больше, чем бездетных, следовательно, <b/>наличие детей негативно влияет на возврат кредита в срок</b>;

2. семейным положением и возвратом кредита в срок - зависимость есть, чаще всего становятся должниками люди с семейным положением <b/>не женат / не замужем</b>;

3. уровнем дохода и возвратом кредита в срок - выраженная зависимость между уровнем дохода и возвратом кредита в срок <b/>не установлена</b>;

4. целью кредита и возвратом кредита в срок - зависимость есть, бОльшая доля должников взяла кредит на <b/>автомобиль или образование</b>.


Как результат, вывили зависимость между потециальным долгом и семейным положением, наличием детей и целью кредита. Наличие зависимости между уровнем дохода и возвратом кредита в срок не подтверждено.
